In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-bn2z7ibd
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-bn2z7ibd
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=39f0a8143dec3fcc7b0150d549626f8e7693922818ae0b45550f1adea135ddc9
  Stored in directory: /tmp/pip-ephem-wheel-cache-t6wqbemh/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
import torch
from tqdm import tqdm
import os
import clip
from PIL import Image
import pandas as pd
import numpy as np
from google.colab import drive

#loading CLIP MODEL and preprocessing function
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device = device)

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 59.0MiB/s]


In [4]:
drive.mount('/content/drive')
DATA_PATH = "/content/drive/MyDrive"

csv_file = DATA_PATH+'/required_dataset/styles2.csv'
df = pd.read_csv(csv_file)
df

Mounted at /content/drive


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,Unnamed: 10,Unnamed: 11
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,NaN,NaN
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,NaN,NaN
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,NaN,NaN
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,NaN,NaN
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
44442,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013,Casual,Gas Men Caddy Casual Shoe,NaN,NaN
44443,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011,Casual,Lotto Men's Soccer Track Flip Flop,NaN,NaN
44444,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011,Casual,Puma Men Graphic Stellar Blue Tshirt,NaN,NaN
44445,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017,Casual,Rasasi Women Blue Lady Perfume,NaN,NaN


In [8]:
#defining image paths and files
image_folder = DATA_PATH+'/required_dataset/images'
embeddings = {}
batch_size = 32

In [7]:
#function to combine text features from multiple columns
def create_text_description(row):
  columns = [
      str(row['gender']),
      str(row['masterCategory']),
      str(row['subCategory']),
      str(row['articleType']),
      str(row['baseColour']),
      str(row['season']),
      str(row['year']),
      str(row['usage']),
      str(row['productDisplayName'])
  ]
  #concatinating all relevant columns into single description
  #as join only supports string so converted all column values to str to avoid null and integer data types
  return ' '.join(columns)

In [6]:
#iterating through CSV rows to generate embeddings for each ID
for index, row in tqdm(df.iterrows()):
  image_id = str(row['id'])
  #combine multiple columns into one descriptive text
  text_description = create_text_description(row)

  #process the text to get text embeddings
  text_input = clip.tokenize([text_description]).to(device)
  with torch.no_grad():
    text_embedding = model.encode_text(text_input).cpu().numpy()

  #load and preprocess image corresponding to 'id'
  image_path = os.path.join(image_folder, f"{image_id}.jpg")
  if os.path.exists(image_path):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
      image_embedding = model.encode_image(image).cpu().numpy()

  else:
    print(f"Image not found for ID: {image_id}")
    image_embedding = None

  #store embeddings in a dictionary
  embeddings[image_id] = {
      'text_embedding': text_embedding,
      'image_embedding': image_embedding
  }


1it [02:01, 121.15s/it]

Image not found for ID: 15970


150it [05:07,  2.05s/it]


KeyboardInterrupt: 

In [ ]:
#preprocessing the image
image_path = (DATA_PATH+'/dataset/random_test_images/lehenga.png')

#.to(device) method moves the image tensor to the specified computing device either CPU or GPU
image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

In [ ]:
#Encode the  image into CLIP's embedding space
with torch.no_grad():
  image_features = model.encode_image(image)

#normalize the calculated embeddings
image_features /=  image_features.norm(dim = -1, keepdim = True)